In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('../../')))
import logging
# Logging setup
# handler = logging.StreamHandler(sys.stdout)
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)

# module_logger = logging.getLogger()
# module_logger.addHandler(handler)
# module_logger.setLevel(logging.DEBUG)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from app.base import LanguageModel, Chai as Chain, Prompt, Comp as Component, Thre as Threads, Placeholder
import pandas as pd
import dtale

DEBUG:matplotlib:matplotlib data path: /Users/fuad/opt/miniconda3/envs/lexiflow/lib/python3.10/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/Users/fuad/.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is darwin
DEBUG:matplotlib:CACHEDIR=/Users/fuad/.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /Users/fuad/.matplotlib/fontlist-v330.json


# tests

In [4]:
# pd.DataFrame(chain.memory)
# dtale.show(pd.DataFrame(lm3.memory))

# parsers
# list as output
# routers
# retriever


In [6]:
# add placeholders and import 

chain1 = Chain([Component(LanguageModel(name="OpenAI1"), 
                          Prompt("tell me a joke about {subject}.", Placeholder('joke'))), 
                Component(LanguageModel(name="Google1"), 
                          Prompt("what do you think of this joke: {joke}?", Placeholder('other'))), 
                Component(LanguageModel(name="Microsoft1"), 
                          Prompt("can you explain why this joke is funny: {joke}?", Placeholder('subject')))])

chain2 = Chain([Component(LanguageModel(name="OpenAI2"), Prompt("tell me a joke about {subject}.", Placeholder('joke'))), 
                Component(LanguageModel(name="Google2"), Prompt("what do you think of this joke: {joke}?", Placeholder('think'))), 
                Component(LanguageModel(name="Microsoft2"), Prompt("can you explain why this joke is funny: {think}?", Placeholder('lala')))])

threads1 = Threads([Component(LanguageModel(name="OpenAI2"), Prompt("tell me a joke about {lala}.", Placeholder('joke'))), 
                Component(LanguageModel(name="Google2"), Prompt("what do you think of this joke: {else}?", Placeholder('think'))), 
                Component(LanguageModel(name="Microsoft2"), Prompt("can you explain why this joke is funny: {think}?", Placeholder('kaka')))])

component = Component(LanguageModel(name="OpenAI2"), Prompt("tell me a joke about {other}.", Placeholder('joke')))

# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders

## Component

In [7]:
initial_placeholders = {"other": Placeholder(name='other', string_content='computer'), 
                        'something': Placeholder(name='something', string_content='something')}
component.run(initial_placeholders)

DEBUG:app.base.executable:Running object: {'components': None, 'memory': [], 'model': LanguageModel(name='OpenAI2', memory=[]), 'prompt': Prompt(template='tell me a joke about {other}.', output=Placeholder(name='joke', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content=None, list_content=None), inputs=['other'], filled_prompt=None)}


{'joke': Placeholder(name='joke', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of OpenAI2: to (tell me a joke about computer.)', list_content=None)}

In [8]:
component.memory

['Response of OpenAI2: to (tell me a joke about computer.)']

## Chain

In [9]:
# Run the chain
initial_placeholders = {"subject": Placeholder(name='subject', string_content='computer'), 
                        'something': Placeholder(name='something', string_content='something')}
chain1.run(initial_placeholders)

DEBUG:app.base.executable:Running object: {'components': [<app.base.executable.Comp object at 0x7f94e51e6740>, <app.base.executable.Comp object at 0x7f94e51e6800>, <app.base.executable.Comp object at 0x7f94e51e6830>], 'memory': []}
DEBUG:app.base.executable:Running object: {'components': None, 'memory': [], 'model': LanguageModel(name='OpenAI1', memory=[]), 'prompt': Prompt(template='tell me a joke about {subject}.', output=Placeholder(name='joke', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content=None, list_content=None), inputs=['subject'], filled_prompt=None)}
DEBUG:app.base.executable:Running object: {'components': None, 'memory': [], 'model': LanguageModel(name='Google1', memory=[]), 'prompt': Prompt(template='what do you think of this joke: {joke}?', output=Placeholder(name='other', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content=None, list_content=None), inpu

{'subject': Placeholder(name='subject', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?)', list_content=None),
 'something': Placeholder(name='something', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='something', list_content=None),
 'joke': Placeholder(name='joke', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of OpenAI1: to (tell me a joke about computer.)', list_content=None),
 'other': Placeholder(name='other', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of Google1: to (what do you think of this joke: Response of OpenAI1: to (tell me a joke about computer.)?)', list_content=None)}

## Chain of chains

In [10]:
chain3 = Chain([chain1, chain2, threads1])
initial_placeholders = {"subject": Placeholder(name='subject', string_content='computer'), 
                        'something': Placeholder(name='something', string_content='something')}
chain1.run(initial_placeholders)

DEBUG:app.base.executable:Running object: {'components': [<app.base.executable.Comp object at 0x7f94e51e6740>, <app.base.executable.Comp object at 0x7f94e51e6800>, <app.base.executable.Comp object at 0x7f94e51e6830>], 'memory': [{'joke': Placeholder(name='joke', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of OpenAI1: to (tell me a joke about computer.)', list_content=None)}, {'other': Placeholder(name='other', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of Google1: to (what do you think of this joke: Response of OpenAI1: to (tell me a joke about computer.)?)', list_content=None)}, {'subject': Placeholder(name='subject', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke a

{'subject': Placeholder(name='subject', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?)', list_content=None),
 'something': Placeholder(name='something', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='something', list_content=None),
 'joke': Placeholder(name='joke', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of OpenAI1: to (tell me a joke about computer.)', list_content=None),
 'other': Placeholder(name='other', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of Google1: to (what do you think of this joke: Response of OpenAI1: to (tell me a joke about computer.)?)', list_content=None)}

## Threads

In [11]:
initial_placeholders = {"subject": Placeholder(name='subject', string_content='computer'), 
                        'lala': Placeholder(name='lala', string_content='something'), 
                       'think': Placeholder(name='think', string_content='something'),
                       'else': Placeholder(name='else', string_content='something')}
threads1.run(initial_placeholders)

DEBUG:app.base.executable:Running object: {'components': [<app.base.executable.Comp object at 0x7f94df582bc0>, <app.base.executable.Comp object at 0x7f94df583be0>, <app.base.executable.Comp object at 0x7f94df583910>], 'memory': []}
DEBUG:app.base.executable:Running object: {'components': None, 'memory': [], 'model': LanguageModel(name='OpenAI2', memory=[]), 'prompt': Prompt(template='tell me a joke about {lala}.', output=Placeholder(name='joke', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content=None, list_content=None), inputs=['lala'], filled_prompt=None)}
DEBUG:app.base.executable:Running object: {'components': None, 'memory': [], 'model': LanguageModel(name='Google2', memory=[]), 'prompt': Prompt(template='what do you think of this joke: {else}?', output=Placeholder(name='think', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content=None, list_content=None), inputs=['e

{'subject': Placeholder(name='subject', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='computer', list_content=None),
 'lala': Placeholder(name='lala', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='something', list_content=None),
 'think': Placeholder(name='think', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of Google2: to (what do you think of this joke: something?)', list_content=None),
 'else': Placeholder(name='else', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='something', list_content=None),
 'joke': Placeholder(name='joke', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of OpenAI2: to (tell me a joke about something.)', list_content=None),
 

# More complex

In [14]:
# add placeholders and import 
threads1 = Threads([Component(LanguageModel(name="OpenAI2"), Prompt("tell me a joke about {lala}.", Placeholder('joke'))), 
                Component(LanguageModel(name="Google2"), Prompt("what do you think of this joke: {something}?", Placeholder('think'))), 
                Component(LanguageModel(name="Microsoft2"), Prompt("can you explain why this joke is funny: {think}?", Placeholder('kaka')))])

chain2 = Chain([Component(LanguageModel(name="OpenAI2"), Prompt("tell me a joke about {subject}.", Placeholder('joke'))), 
                Component(LanguageModel(name="Google2"), Prompt("what do you think of this joke: {joke}?", Placeholder('think'))), 
                Component(LanguageModel(name="Microsoft2"), Prompt("can you explain why this joke is funny: {think}?", Placeholder('lala'))),
                threads1
               ])

chain1 = Chain([Component(LanguageModel(name="OpenAI1"), 
                          Prompt("tell me a joke about {subject}.", Placeholder('joke'))), 
                Component(LanguageModel(name="Google1"), 
                          Prompt("what do you think of this joke: {joke}?", Placeholder('other'))), 
                Component(LanguageModel(name="Microsoft1"), 
                          Prompt("can you explain why this joke is funny: {joke}?", Placeholder('subject'))),
                chain2
               ])

component = Component(LanguageModel(name="OpenAI2"), Prompt("tell me a joke about {other}.", Placeholder('joke')))

In [15]:

# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders

chain3 = Chain([chain1, chain2, threads1])
initial_placeholders = {"subject": Placeholder(name='subject', string_content='computer'), 
                        'something': Placeholder(name='something', string_content='something')}
chain1.run(initial_placeholders)

DEBUG:app.base.executable:Running object: {'components': [<app.base.executable.Comp object at 0x7f94e5441870>, <app.base.executable.Comp object at 0x7f94e53c90c0>, <app.base.executable.Comp object at 0x7f94e53cbee0>, <app.base.executable.Chai object at 0x7f94e53ca020>], 'memory': []}
DEBUG:app.base.executable:Running object: {'components': None, 'memory': [], 'model': LanguageModel(name='OpenAI1', memory=[]), 'prompt': Prompt(template='tell me a joke about {subject}.', output=Placeholder(name='joke', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content=None, list_content=None), inputs=['subject'], filled_prompt=None)}
DEBUG:app.base.executable:Running object: {'components': None, 'memory': [], 'model': LanguageModel(name='Google1', memory=[]), 'prompt': Prompt(template='what do you think of this joke: {joke}?', output=Placeholder(name='other', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa

{'subject': Placeholder(name='subject', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?)', list_content=None),
 'something': Placeholder(name='something', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='something', list_content=None),
 'joke': Placeholder(name='joke', type='identity', choice='all', listify_func=<function Placeholder.<lambda> at 0x7f94dfa48670>, string_content='Response of OpenAI2: to (tell me a joke about Response of Microsoft2: to (can you explain why this joke is funny: Response of Google2: to (what do you think of this joke: Response of OpenAI2: to (tell me a joke about Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?).)?)?).)', list_content=N